In [0]:
import tensorflow as tf
import torchaudio
import sys,os
import pandas as pd
import numpy as np
from numpy.random import randint
import matplotlib.pyplot as plt
import librosa
import librosa.display
import re

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# models
from sklearn import linear_model, naive_bayes, neighbors
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 
from sklearn.linear_model import Perceptron
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D

# stat libraries
from scipy import stats

# Libraries for the evaluation
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
%matplotlib inline

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.nearest_centroid module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/Speech Project/recordings.zip", 'r')
zip_ref.extractall("/content")
zip_ref.close()

In [0]:
speakers_clean = pd.read_csv('/content/drive/My Drive/Speech Project/speakers_clean.csv')
speakers_clean

,age,age_onset,birthplace,filename,native_language,sex,speakerid,country,file_missing?,continent
0,27.0,9.0,"virginia, south africa",afrikaans1,afrikaans,female,1,south africa,False,Africa
1,40.0,5.0,"pretoria, south africa",afrikaans2,afrikaans,male,2,south africa,False,Africa
2,43.0,4.0,"pretoria, transvaal, south africa",afrikaans3,afrikaans,male,418,south africa,False,Africa
3,26.0,8.0,"pretoria, south africa",afrikaans4,afrikaans,male,1159,south africa,False,Africa
4,19.0,6.0,"cape town, south africa",afrikaans5,afrikaans,male,1432,south africa,False,Africa
...,...,...,...,...,...,...,...,...,...,...
2135,46.0,5.0,"lagos, nigeria",yoruba3,yoruba,female,766,nigeria,False,Africa
2136,46.0,12.0,"lagos, nigeria",yoruba4,yoruba,male,851,nigeria,False,Africa
2137,47.0,2.0,"ibadan, nigeria",yoruba5,yoruba,female,2023,nigeria,False,Africa
2138,31.0,1.0,"bethel, alaska, usa",yupik1,yupik,female,571,usa,False,North America


#  Data Loading & Feature Extraction

In [0]:
sample_rate = 16000
def extract_features(audio):
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
 
    return mfccs

In [0]:
root = "./recordings/"
part1 = pd.DataFrame(columns=['filename', 'features'])    
part1_dict = {}

i = 0
max_pad_len = 3500
maxlen = 0

for path, subdirs, files in os.walk(root):
    for name in files:
        if i == 400:
            break
        filename = name
        sound, sample_rate = librosa.load(os.path.join(path, name))
        data = extract_features(sound)
        
        if maxlen == 0 or maxlen < data.shape[1]:
            maxlen = data.shape[1]
            
        # zero-pad the mfccs features in order to have all compatible shapes for input of the CNN.
        # max_pad_len is the biggest number of audio frames
        # obtained by extracting features (mfccs) from all the audio files.
        #pad_width = max_pad_len - data.shape[1]
        #data = np.pad(data, pad_width=((0,0), (0, pad_width)), mode='constant')
        
        print(i)
        part1_dict[filename] = [data]
        i += 1

print(maxlen)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
import json
import numpy
from json import JSONEncoder

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)
  
with open('part1.json', 'w') as fp:
    json.dump(part1_dict, fp, cls=NumpyArrayEncoder)

In [0]:
import json
import numpy
from json import JSONEncoder

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)
  
with open('part2.json', 'w') as fp:
    json.dump(part2_dict, fp, cls=NumpyArrayEncoder)

NameError: ignored

In [0]:
import json
import numpy
from json import JSONEncoder

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)
        
with open('part3.json', 'w') as fp:
    json.dump(part3_dict, fp, cls=NumpyArrayEncoder)

In [0]:
with open('part4.json', 'w') as fp:
    json.dump(part4_dict, fp, cls=NumpyArrayEncoder)

In [0]:
with open('part5.json', 'w') as fp:
    json.dump(part5_dict, fp, cls=NumpyArrayEncoder)

In [0]:
!ls

drive  __MACOSX  part1.json  part2.json  recordings  sample_data


In [0]:
import json
with open('part1.json') as f:
  part1_test = json.load(f)

part1_test

In [0]:
print(part1_test)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
part1_df = pd.DataFrame.from_dict(part1_dict, orient='index', columns=['features'])

In [0]:
part1_df.to_csv('part1_df.csv')

In [0]:
part1_df.loc[:, 'features'] = part1_df['features'].apply(list)

In [0]:
part1_df_test = pd.read_csv('part1_df.csv')

In [0]:
max_part1 = 3666
max_part2 = 3334
max_part3 = 2800
max_part4 = 3348
part1.to_csv('feature_part1.csv')

In [0]:
root = "./recordings/"
i = 400
maxlen = 0
part2_dict = {}

for path, subdirs, files in os.walk(root):
    for name in files[400:800]:
        filename = name
        sound, sample_rate = librosa.load(os.path.join(path, name))
        data = extract_features(sound)
        if maxlen == 0 or maxlen < data.shape[1]:
            maxlen = data.shape[1]
        print(i)
        part2_dict[filename] = [data]
        i += 1
        
print(maxlen)

400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649


##-----------------------Start Here---------------------------

In [0]:
# read json to csv
part1 = pd.read_json('/content/drive/My Drive/Speech Project/part1.json', orient='index')
part2 = pd.read_json('/content/drive/My Drive/Speech Project/part2.json', orient='index')
part3 = pd.read_json('/content/drive/My Drive/Speech Project/part3.json', orient='index')
part4 = pd.read_json('/content/drive/My Drive/Speech Project/part4.json', orient='index')
part5 = pd.read_json('/content/drive/My Drive/Speech Project/part5.json', orient='index')

In [0]:
# concat all
features = pd.concat([part1, part2, part3, part4, part5]).reset_index()
# change column names
features_clean = features.rename(columns={"index": "filename", 0: "features"})
# edit file names
features_clean['filename'] = features_clean['filename'].apply(lambda x: x[:-4])

In [0]:
# read speaker info csv
speakers_clean = pd.read_csv('/content/drive/My Drive/Speech Project/speakers_clean.csv')

In [0]:
# combine features with speakers
speaker_and_feature = speakers_clean.merge(features_clean, left_on='filename', right_on='filename')

In [0]:
len(speaker_and_feature.features[0])

40

# Train / Test split

In [0]:
from sklearn.model_selection import train_test_split
#speaker_and_feature = pd.read_csv('speaker_and_feature.csv')

In [0]:

# use countries of origin as proxies of their races
# choose usa(76.5% white), canada, and europe speakers as the training set

train_countries = ['usa', 'canada']
train_df = pd.concat([speaker_and_feature[speaker_and_feature['country'].isin(train_countries)], 
                     speaker_and_feature[speaker_and_feature['continent'] == 'Europe']])
# obtain test set for white speakers by a 85-15 split 
train_df, test_df = train_test_split(train_df, test_size=.15)

# we use african-country speakers and asian-country speakers only as the test sets
test_african_df = speaker_and_feature[speaker_and_feature['continent'] == 'Africa']
test_asian_df = speaker_and_feature[speaker_and_feature['continent'] == 'Asia']

print('Training size (total)', len(train_df))
print('Training size (female)', len(train_df[train_df['sex'] == 'female']))
print('Training size (male)', len(train_df[train_df['sex'] == 'male']))

print('Test1 size (total)', len(test_df))
print('Test1 size (female)', len(test_df[test_df['sex'] == 'female']))
print('Test1 size (male)', len(test_df[test_df['sex'] == 'male']))

print('\nTest2 size (african, all)', len(test_african_df))
print('Test2 size (african, female)', len(test_african_df[test_african_df['sex'] == 'female']))
print('Test2 size (african, male)', len(test_african_df[test_african_df['sex'] == 'male']))

print('\nTest3 size (asian)', len(test_asian_df))
print('Test3 size (asian, female)', len(test_asian_df[test_asian_df['sex'] == 'female']))
print('Test3 size (asian, male)', len(test_asian_df[test_asian_df['sex'] == 'male']))

Training size (total) 711
Training size (female) 345
Training size (male) 366
Test1 size (total) 126
Test1 size (female) 51
Test1 size (male) 75

Test2 size (african, all) 189
Test2 size (african, female) 72
Test2 size (african, male) 117

Test3 size (asian) 525
Test3 size (asian, female) 271
Test3 size (asian, male) 253


In [0]:
train_df.head()

,age,age_onset,birthplace,filename,native_language,sex,speakerid,country,file_missing?,continent,features
744,18.0,0.0,"boston, massachusetts, usa",english76,english,female,139,usa,False,North America,"[[-545.5300332271895, -544.5147273480371, -545..."
399,26.0,0.0,"myrtle beach, south carolina, usa",english214,english,male,737,usa,False,North America,"[[-397.87663064287074, -396.7454699072496, -39..."
608,27.0,0.0,"ottawa, ontario, canada",english464,english,male,1618,canada,False,North America,"[[-493.55471019878087, -492.26183630345247, -4..."
674,22.0,0.0,"plymouth, massachusetts, usa",english535,english,female,1955,usa,False,North America,"[[-476.93032868169723, -472.61264640519545, -4..."
970,20.0,8.0,"cosenza, italy",italian14,italian,male,761,italy,False,Europe,"[[-569.3366800202974, -569.3366800202974, -569..."


In [0]:
# clean data
test_asian_df.loc[test_asian_df['sex'] == 'famale', 'sex'] = 'female'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [0]:
X_train = np.array(train_df.features.apply(lambda x: np.mean(np.array(x), axis=1)).to_list())
X_test = np.array(test_df.features.apply(lambda x: np.mean(np.array(x), axis=1)).to_list())
X_test_african = np.array(test_african_df.features.apply(lambda x: np.mean(np.array(x), axis=1)).to_list())
X_test_asian = np.array(test_asian_df.features.apply(lambda x: np.mean(np.array(x), axis=1)).to_list())

# 0=female, 1=male
le = LabelEncoder()
y_train = le.fit_transform(np.array(train_df.sex.tolist()))
y_test = le.fit_transform(np.array(test_df.sex.tolist()))
y_test_african = le.fit_transform(np.array(test_african_df.sex.tolist()))
y_test_asian = le.fit_transform(np.array(test_asian_df.sex.tolist()))

print('Training feature shape:', X_train.shape)
print('Training label shape:', y_train.shape)
print('Testing (baseline) feature shape:', X_test.shape)
print('Testing (baseline) label shape:', y_test.shape)
print('\nTesting feature (african) shape:', X_test_african.shape)
print('Testing label (african) shape:', y_test_african.shape)
print('\nTesting feature (asian) shape:', X_test_asian.shape)
print('Testing label (asian) shape:', y_test_asian.shape)

Training feature shape: (711, 40)
Training label shape: (711,)
Testing (baseline) feature shape: (126, 40)
Testing (baseline) label shape: (126,)

Testing feature (african) shape: (189, 40)
Testing label (african) shape: (189,)

Testing feature (african) shape: (525, 40)
Testing label (african) shape: (525,)


# KNN

In [0]:
# hyperparameter tuning
for n in range(1,21):
  knn = neighbors.KNeighborsClassifier(n, weights='distance')
  print('N={}, cross validation accuracy='.format(n), np.mean(cross_val_score(knn, X_train, y_train, scoring='accuracy', cv=10)))

N=1, cross validation accuracy= 0.8368348982785603
N=2, cross validation accuracy= 0.8368348982785603
N=3, cross validation accuracy= 0.8481220657276995
N=4, cross validation accuracy= 0.8565336463223787
N=5, cross validation accuracy= 0.8537754303599373
N=6, cross validation accuracy= 0.8565727699530516
N=7, cross validation accuracy= 0.8523474178403756
N=8, cross validation accuracy= 0.8607785602503911
N=9, cross validation accuracy= 0.8509389671361502
N=10, cross validation accuracy= 0.8678403755868545
N=11, cross validation accuracy= 0.842488262910798
N=12, cross validation accuracy= 0.8565532081377152
N=13, cross validation accuracy= 0.853755868544601
N=14, cross validation accuracy= 0.8593114241001565
N=15, cross validation accuracy= 0.8494718309859154
N=16, cross validation accuracy= 0.8508802816901408
N=17, cross validation accuracy= 0.8438771517996869
N=18, cross validation accuracy= 0.8509194053208138
N=19, cross validation accuracy= 0.8438771517996869
N=20, cross validation 

In [0]:
# choose n=8
knn = neighbors.KNeighborsClassifier(8, weights='distance')
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
print("Confusion Matrix (Baseline):")
print(confusion_matrix(y_test, y_pred))
print("\nAccuracy: ", accuracy_score(y_test, y_pred))
print("\nPrecision: ", precision_score(y_test, y_pred))
print("\nRecall: ", recall_score(y_test, y_pred))

y_pred_african = knn.predict(X_test_african)
print("Confusion Matrix (African):")
print(confusion_matrix(y_test_african, y_pred_african))
print("\nAccuracy: ", accuracy_score(y_test_african, y_pred_african))
print("\nPrecision: ", precision_score(y_test_african, y_pred_african))
print("\nRecall: ", recall_score(y_test_african, y_pred_african))

y_pred_asian = knn.predict(X_test_asian)
print("\nConfusion Matrix (Asian):")
print(confusion_matrix(y_test_asian, y_pred_asian))
print("\nAccuracy: ", accuracy_score(y_test_asian, y_pred_asian))
print("\nPrecision: ", precision_score(y_test_asian, y_pred_asian))
print("\nRecall: ", recall_score(y_test_asian, y_pred_asian))

Confusion Matrix (Baseline):
[[45 18]
 [ 4 59]]

Accuracy:  0.8253968253968254

Precision:  0.7662337662337663

Recall:  0.9365079365079365
Confusion Matrix (African):
[[49 23]
 [28 89]]

Accuracy:  0.7301587301587301

Precision:  0.7946428571428571

Recall:  0.7606837606837606

Confusion Matrix (Asian):
[[196  76]
 [ 36 217]]

Accuracy:  0.7866666666666666

Precision:  0.7406143344709898

Recall:  0.857707509881423


Notice that the confusion matrix here at (0,0), (1,0), (0,1), (1,1) are TN, FN, FP, TP respectively.  
Recall that we expect to have more False Positives (true: female, predicted: male) for Africans and more False Negatives (true: male, predicted: female) for Asians.  
In this case, the False Positive Rate(FPR) for the baseline is 18/(18+45) = 0.286. The FPR for Africans is 23/(23+49) = 0.319 and the FPR for Asians is 76/(76+196) = 0.279. The False Negative Rate(FNR) for the baseline is 4/(4+59) = 0.063, while the FNR for Africans is 28/(28+89) = 0.239 and for Asians is 30/(30+223) = 0.119.  
###Asians are lower in both (compared to African counterparts), so it's 50% consistent with our hypothesis for KNN.

# Naive Bayes

In [0]:
nb = naive_bayes.GaussianNB();
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)
print("Confusion Matrix (Baseline):")
print(confusion_matrix(y_test, y_pred))
print("\nAccuracy: ", accuracy_score(y_test, y_pred))
print("\nPrecision: ", precision_score(y_test, y_pred))
print("\nRecall: ", recall_score(y_test, y_pred))

y_pred_african = nb.predict(X_test_african)
print("Confusion Matrix (African):")
print(confusion_matrix(y_test_african, y_pred_african))
print("\nAccuracy: ", accuracy_score(y_test_african, y_pred_african))
print("\nPrecision: ", precision_score(y_test_african, y_pred_african))
print("\nRecall: ", recall_score(y_test_african, y_pred_african))

y_pred_asian = nb.predict(X_test_asian)
print("\nConfusion Matrix (Asian):")
print(confusion_matrix(y_test_asian, y_pred_asian))
print("\nAccuracy: ", accuracy_score(y_test_asian, y_pred_asian))
print("\nPrecision: ", precision_score(y_test_asian, y_pred_asian))
print("\nRecall: ", recall_score(y_test_asian, y_pred_asian))

Confusion Matrix (Baseline):
[[53 10]
 [ 4 59]]

Accuracy:  0.8888888888888888

Precision:  0.855072463768116

Recall:  0.9365079365079365
Confusion Matrix (African):
[[ 60  12]
 [ 12 105]]

Accuracy:  0.873015873015873

Precision:  0.8974358974358975

Recall:  0.8974358974358975

Confusion Matrix (Asian):
[[244  28]
 [ 21 232]]

Accuracy:  0.9066666666666666

Precision:  0.8923076923076924

Recall:  0.9169960474308301


In this case, the False Positive Rate(FPR) for Africans is 12/(12+60) = 0.167 and the FPR for Asians is 28/(28+244) = 0.103. The False Negative Rate(FNR) for Africans is 12/(12+105) = 0.102 and for Asians is 21/(21+232) = 0.083. 

The baselines are FPR = 0.159, and FNR = 0.063
###Asians are lower in both, so 50% consistent with our hypothesis for Naive Bayes.

# Support Vector Machine

In [0]:
# hyperparameter tuning

C = [0.1, 1, 10, 1e2, 1e3]
Gamma = ['scale', 'auto']
Kernel = ['poly', 'rbf']

for c in C:
  for gamma in Gamma:
    for kernel in Kernel:
      svc = svm.SVC(C=c, kernel=kernel, gamma=gamma)
      print('The cv error for C={}'.format(c), 'gamma={}'.format(gamma), 'kernel={}'.format(kernel), 'is', 
            1-np.mean(cross_val_score(svc, X_train, y_train, scoring='accuracy')))

The cv error for C=0.1 gamma=scale kernel=poly is 0.4683541810302374
The cv error for C=0.1 gamma=scale kernel=rbf is 0.4683541810302374
The cv error for C=0.1 gamma=auto kernel=poly is 0.08865359992120558
The cv error for C=0.1 gamma=auto kernel=rbf is 0.4683541810302374
The cv error for C=1 gamma=scale kernel=poly is 0.12800157588889982
The cv error for C=1 gamma=scale kernel=rbf is 0.15049738993400974
The cv error for C=1 gamma=auto kernel=poly is 0.08865359992120558
The cv error for C=1 gamma=auto kernel=rbf is 0.3839850290554516
The cv error for C=10 gamma=scale kernel=poly is 0.07454939426770402
The cv error for C=10 gamma=scale kernel=rbf is 0.07454939426770402
The cv error for C=10 gamma=auto kernel=poly is 0.08865359992120558
The cv error for C=10 gamma=auto kernel=rbf is 0.3839850290554516
The cv error for C=100.0 gamma=scale kernel=poly is 0.07875504776913222
The cv error for C=100.0 gamma=scale kernel=rbf is 0.08156209987195917
The cv error for C=100.0 gamma=auto kernel=pol

In [0]:
# C=10, gamma=scale, kernel=poly
svc = svm.SVC(C=10, kernel='poly', gamma='scale')
svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)
print("Confusion Matrix (Baseline):")
print(confusion_matrix(y_test, y_pred))
print("\nAccuracy: ", accuracy_score(y_test, y_pred))
print("\nPrecision: ", precision_score(y_test, y_pred))
print("\nRecall: ", recall_score(y_test, y_pred))

y_pred_african = svc.predict(X_test_african)
print("Confusion Matrix (African):")
print(confusion_matrix(y_test_african, y_pred_african))
print("\nAccuracy: ", accuracy_score(y_test_african, y_pred_african))
print("\nPrecision: ", precision_score(y_test_african, y_pred_african))
print("\nRecall: ", recall_score(y_test_african, y_pred_african))

y_pred_asian = svc.predict(X_test_asian)
print("\nConfusion Matrix (Asian):")
print(confusion_matrix(y_test_asian, y_pred_asian))
print("\nAccuracy: ", accuracy_score(y_test_asian, y_pred_asian))
print("\nPrecision: ", precision_score(y_test_asian, y_pred_asian))
print("\nRecall: ", recall_score(y_test_asian, y_pred_asian))

Confusion Matrix (Baseline):
[[54  9]
 [ 5 58]]

Accuracy:  0.8888888888888888

Precision:  0.8656716417910447

Recall:  0.9206349206349206
Confusion Matrix (African):
[[ 62  10]
 [  7 110]]

Accuracy:  0.91005291005291

Precision:  0.9166666666666666

Recall:  0.9401709401709402

Confusion Matrix (Asian):
[[255  17]
 [ 15 238]]

Accuracy:  0.939047619047619

Precision:  0.9333333333333333

Recall:  0.9407114624505929


In this case, the False Positive Rate(FPR) for Africans is 10/(10+62) = 0.139 and the FPR for Asians is 17/(17+255) = 0.0625. The False Negative Rate(FNR) for Africans is 7/(7+110) = 0.060 and for Asians is 15/(15+238) = 0.059

Baselines are FPR = 0.143, FNR = 0.079.
###Asians are lower in both, so it's 50% consistent with the hypothesis.

# Linear Perceptron

In [0]:
# hyperparameter tuning
alphas = [0.00001,0.00005,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1,5]
for alpha in alphas:
    perceptron = Perceptron(penalty='l2', alpha=alpha, shuffle=True) # training data are shuffled at each epoch.
    perceptron.fit(X_train, y_train)
    print('alpha={}, cross validation accuracy='.format(alpha), np.mean(cross_val_score(perceptron, X_train, y_train, scoring='accuracy', cv=10)))

alpha=1e-05, cross validation accuracy= 0.8199334898278561
alpha=5e-05, cross validation accuracy= 0.8707550860719875
alpha=0.0001, cross validation accuracy= 0.8171752738654146
alpha=0.0005, cross validation accuracy= 0.6951095461658843
alpha=0.001, cross validation accuracy= 0.7260758998435055
alpha=0.005, cross validation accuracy= 0.5190532081377152
alpha=0.01, cross validation accuracy= 0.49088419405320816
alpha=0.05, cross validation accuracy= 0.5091940532081377
alpha=0.1, cross validation accuracy= 0.5007433489827855
alpha=0.5, cross validation accuracy= 0.49792644757433485
alpha=1, cross validation accuracy= 0.49370109546165875
alpha=5, cross validation accuracy= 0.49370109546165875


In [0]:
perceptron = Perceptron(penalty='l2', alpha=5e-5, shuffle=True) # training data are shuffled at each epoch.
perceptron.fit(X_train, y_train)

y_pred = perceptron.predict(X_test)
print("Confusion Matrix (Baseline):")
print(confusion_matrix(y_test, y_pred))
print("\nAccuracy: ", accuracy_score(y_test, y_pred))
print("\nPrecision: ", precision_score(y_test, y_pred))
print("\nRecall: ", recall_score(y_test, y_pred))

y_pred_african = perceptron.predict(X_test_african)
print("Confusion Matrix (African):")
print(confusion_matrix(y_test_african, y_pred_african))
print("\nAccuracy: ", accuracy_score(y_test_african, y_pred_african))
print("\nPrecision: ", precision_score(y_test_african, y_pred_african))
print("\nRecall: ", recall_score(y_test_african, y_pred_african))

y_pred_asian = perceptron.predict(X_test_asian)
print("\nConfusion Matrix (Asian):")
print(confusion_matrix(y_test_asian, y_pred_asian))
print("\nAccuracy: ", accuracy_score(y_test_asian, y_pred_asian))
print("\nPrecision: ", precision_score(y_test_asian, y_pred_asian))
print("\nRecall: ", recall_score(y_test_asian, y_pred_asian))

Confusion Matrix (Baseline):
[[60  3]
 [19 44]]

Accuracy:  0.8253968253968254

Precision:  0.9361702127659575

Recall:  0.6984126984126984
Confusion Matrix (African):
[[70  2]
 [38 79]]

Accuracy:  0.7883597883597884

Precision:  0.9753086419753086

Recall:  0.6752136752136753

Confusion Matrix (Asian):
[[267   5]
 [ 78 175]]

Accuracy:  0.8419047619047619

Precision:  0.9722222222222222

Recall:  0.691699604743083


Precisions are very high and recalls are very low in this case, the False Positive Rate(FPR) for Africans is 2/72 = 0.028 and the FPR for Asians is 5/272=0.018. The False Negative Rate(FNR) for Africans is 38/(38+79) = 0.325 and for Asians is 78/(78+175) = 0.308.  

Baselines are FPR = 0.048, FNR = 0.302.
###Asians are lower in both, so it's 50% consistent with the hypothesis.  
###But this model is biased towards higher precision and lower recall.

# Logistic Regression

In [0]:
# hyperparameter tuning

C = [0.1, 1, 10, 1e2, 1e3, 1e4]
Solver = ['lbfgs', 'saga']

for c in C:
  for solver in Solver:
    logit = linear_model.LogisticRegression(C=c, solver=solver, max_iter=10000)
    print('The cv error for C={}'.format(c), 'solver={}'.format(solver), 'is', 
          1-np.mean(cross_val_score(logit, X_train, y_train, scoring='accuracy')))

The cv error for C=0.1 solver=lbfgs is 0.08440854919728158
The cv error for C=0.1 solver=saga is 0.08440854919728158
The cv error for C=1 solver=lbfgs is 0.08723529991135626
The cv error for C=1 solver=saga is 0.08580715059588295
The cv error for C=10 solver=lbfgs is 0.08864375061558172
The cv error for C=10 solver=saga is 0.08580715059588295
The cv error for C=100.0 solver=lbfgs is 0.08723529991135626
The cv error for C=100.0 solver=saga is 0.08580715059588295
The cv error for C=1000.0 solver=lbfgs is 0.08723529991135626
The cv error for C=1000.0 solver=saga is 0.08580715059588295
The cv error for C=10000.0 solver=lbfgs is 0.08723529991135626
The cv error for C=10000.0 solver=saga is 0.08580715059588295


In [0]:
# C=0.1, solver=saga

logit = linear_model.LogisticRegression(C=0.1, solver='saga', max_iter=10000, verbose=0)
logit.fit(X_train, y_train)

y_pred = logit.predict(X_test)
print("Confusion Matrix (Baseline):")
print(confusion_matrix(y_test, y_pred))
print("\nAccuracy: ", accuracy_score(y_test, y_pred))
print("\nPrecision: ", precision_score(y_test, y_pred))
print("\nRecall: ", recall_score(y_test, y_pred))

y_pred_african = logit.predict(X_test_african)
print("Confusion Matrix (African):")
print(confusion_matrix(y_test_african, y_pred_african))
print("\nAccuracy: ", accuracy_score(y_test_african, y_pred_african))
print("\nPrecision: ", precision_score(y_test_african, y_pred_african))
print("\nRecall: ", recall_score(y_test_african, y_pred_african))

y_pred_asian = logit.predict(X_test_asian)
print("\nConfusion Matrix (Asian):")
print(confusion_matrix(y_test_asian, y_pred_asian))
print("\nAccuracy: ", accuracy_score(y_test_asian, y_pred_asian))
print("\nPrecision: ", precision_score(y_test_asian, y_pred_asian))
print("\nRecall: ", recall_score(y_test_asian, y_pred_asian))

Confusion Matrix (Baseline):
[[57  6]
 [ 3 60]]

Accuracy:  0.9285714285714286

Precision:  0.9090909090909091

Recall:  0.9523809523809523
Confusion Matrix (African):
[[ 64   8]
 [  9 108]]

Accuracy:  0.91005291005291

Precision:  0.9310344827586207

Recall:  0.9230769230769231

Confusion Matrix (Asian):
[[253  19]
 [ 19 234]]

Accuracy:  0.9276190476190476

Precision:  0.924901185770751

Recall:  0.924901185770751


The False Positive Rate(FPR) for Africans is 8/(8+64) = 0.111 and the FPR for Asians is 19/(19+253) = 0.070. The False Negative Rate(FNR) for Africans is 9/(9+108) = 0.077 and for Asians is 15/(15+238) = 0.075.

Baselines are FPR = 0.095, FNR = 0.048
###Asians are lower in both, so it's 50% consistent with the hypothesis.  



# ANN

In [0]:
# hyperparameter tuning

Activation = ['identity', 'logistic', 'tanh', 'relu']
Solver = ['lbfgs', 'sgd', 'adam']


for activation in Activation:
  for solver in Solver:
    mlp = MLPClassifier(activation=activation, solver=solver, max_iter=10000)
    print('The cv error for activation={}'.format(activation), 'solver={}'.format(solver), 'is', 
          1-np.mean(cross_val_score(mlp, X_train, y_train, scoring='accuracy')))

The cv error for activation=identity solver=lbfgs is 0.5246035654486358
The cv error for activation=identity solver=sgd is 0.07315079286910287
The cv error for activation=identity solver=adam is 0.0872156013001083
The cv error for activation=logistic solver=lbfgs is 0.10407761252831682
The cv error for activation=logistic solver=sgd is 0.07174234216487752
The cv error for activation=logistic solver=adam is 0.07454939426770424
The cv error for activation=tanh solver=lbfgs is 0.09987195902688861
The cv error for activation=tanh solver=sgd is 0.06889589283955488
The cv error for activation=tanh solver=adam is 0.06047473653107471
The cv error for activation=relu solver=lbfgs is 0.4006599034768049
The cv error for activation=relu solver=sgd is 0.08439869989165771
The cv error for activation=relu solver=adam is 0.08161134640007872


In [0]:
# activation=logistic solver=adam

mlp = MLPClassifier(activation='tanh', solver='adam', max_iter=10000)
mlp.fit(X_train, y_train)

y_pred = mlp.predict(X_test)
print("Confusion Matrix (Baseline):")
print(confusion_matrix(y_test, y_pred))
print("\nAccuracy: ", accuracy_score(y_test, y_pred))
print("\nPrecision: ", precision_score(y_test, y_pred))
print("\nRecall: ", recall_score(y_test, y_pred))

y_pred_african = mlp.predict(X_test_african)
print("Confusion Matrix (African):")
print(confusion_matrix(y_test_african, y_pred_african))
print("\nAccuracy: ", accuracy_score(y_test_african, y_pred_african))
print("\nPrecision: ", precision_score(y_test_african, y_pred_african))
print("\nRecall: ", recall_score(y_test_african, y_pred_african))

y_pred_asian = mlp.predict(X_test_asian)
print("\nConfusion Matrix (Asian):")
print(confusion_matrix(y_test_asian, y_pred_asian))
print("\nAccuracy: ", accuracy_score(y_test_asian, y_pred_asian))
print("\nPrecision: ", precision_score(y_test_asian, y_pred_asian))
print("\nRecall: ", recall_score(y_test_asian, y_pred_asian))

Confusion Matrix (Baseline):
[[56  7]
 [ 4 59]]

Accuracy:  0.9126984126984127

Precision:  0.8939393939393939

Recall:  0.9365079365079365
Confusion Matrix (African):
[[ 66   6]
 [ 11 106]]

Accuracy:  0.91005291005291

Precision:  0.9464285714285714

Recall:  0.905982905982906

Confusion Matrix (Asian):
[[257  15]
 [ 19 234]]

Accuracy:  0.9352380952380952

Precision:  0.9397590361445783

Recall:  0.924901185770751


The False Positive Rate(FPR) for Africans is 6/(6+66) = 0.083 and the FPR for Asians is 15/(15+257) = 0.055. The False Negative Rate(FNR) for Africans is 11/(11+106) = 0.094 and for Asians is 19/(19+234) = 0.075.

Baselines are FPR = 0.111, FNR = 0.063.
###Asians are lower in both, so it's 50% consistent with the hypothesis.  

# CNN

In [0]:
train_df, validation_df = train_test_split(train_df, test_size=.2)
X_train_CNN = np.array(train_df.features)
X_validation_CNN = np.array(validation_df.features)
y_train = le.fit_transform(np.array(train_df.sex.tolist()))
y_validation = le.fit_transform(np.array(validation_df.sex.tolist()))
X_test_CNN = np.array(test_df.features)
X_test_african_CNN = np.array(test_african_df.features)
X_test_asian_CNN = np.array(test_asian_df.features)

In [0]:
# pad mfccs and make tensors
import torch
def get_tensor(arr):
    size = len(arr)
    tensor = np.zeros((size,40,3666))
    for i in range(size):
        for j in range(40):
            diff = 3666 - len(arr[i][j])
            tensor[i][j] = np.hstack((arr[i][j], np.zeros(diff)))
    tensor = tensor.reshape(tensor.shape[0], 1, 40, 3666)
    tensor = torch.tensor(tensor, dtype=torch.float)
    print(tensor.shape)
    return tensor
X_train_tensor = get_tensor(X_train_CNN)
X_validation_tensor = get_tensor(X_validation_CNN)
X_test_tensor = get_tensor(X_test_CNN)
X_test_african_tensor = get_tensor(X_test_african_CNN)
X_test_asian_tensor = get_tensor(X_test_asian_CNN)
y_train_tensor = torch.tensor(y_train)
y_validation_tensor = torch.tensor(y_validation)
y_test_tensor = torch.tensor(y_test)
y_test_african_tensor = torch.tensor(y_test_african)
y_test_asian_tensor = torch.tensor(y_test_asian)

torch.Size([148, 1, 40, 3666])
torch.Size([37, 1, 40, 3666])
torch.Size([126, 1, 40, 3666])
torch.Size([189, 1, 40, 3666])
torch.Size([525, 1, 40, 3666])


In [0]:
from torch.utils import data
trainset = data.TensorDataset(X_train_tensor, y_train_tensor)
trainloader = data.DataLoader(trainset)
validationset = data.TensorDataset(X_validation_tensor, y_validation_tensor)
validationloader = data.DataLoader(validationset)
testset = data.TensorDataset(X_test_tensor, y_test_tensor)
testloader = data.DataLoader(testset)
testset_african = data.TensorDataset(X_test_african_tensor, y_test_african_tensor)
testloader_african = data.DataLoader(testset_african)
testset_asian = data.TensorDataset(X_test_asian_tensor, y_test_asian_tensor)
testloader_asian = data.DataLoader(testset_asian)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, ch1, ch2):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(
            # input 40 * 3666
            nn.Conv2d(1, ch1, (2,6), stride=(1,3), padding=0), # output 39 * 1221
            nn.BatchNorm2d(ch1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=3) # 13 * 404
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(ch1, ch2, (3,4), stride=(1,2), padding=(1,0)), # 12 * 201 
            nn.BatchNorm2d(ch2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2)
        )
        self.fc = nn.Linear(ch2 * 6 * 101, 2)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x 

In [0]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()

def train(ch1, ch2, epochs):
    global accuracies
    net = Net(ch1,ch2)
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    losses = []
    for epoch in range(epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):

            inputs, labels = data # data is a list of [inputs, labels]
            optimizer.zero_grad() # zero the parameter gradients

            outputs = net(inputs) # fwd
            l = torch.tensor(labels, dtype=torch.long)
            loss = criterion(outputs, l)
            loss.backward()       # backwd
            optimizer.step()      # optimize

            # print statistics
            running_loss += loss.item()
            if i % 200 == 199:    # print every 200 mini-batches
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
                losses.append(running_loss / 200)
                running_loss = 0.0
    print('Finished Training')
    correct = 0
    total = 0
    with torch.no_grad():
        for data in validationloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print("Validation accuracy for ch1={} and ch2={}: ".format(ch1, ch2), accuracy)
    accuracies.append(accuracy)
    return net, losses

In [0]:
accuracies = []
for ch1 in [4,8,16,32,64]:
    for ch2 in [4,8,16,32,64]:
        train(ch1, ch2, epochs=5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Finished Training
Validation accuracy for ch1=4 and ch2=4:  86.48648648648648
Finished Training
Validation accuracy for ch1=4 and ch2=8:  70.27027027027027
Finished Training
Validation accuracy for ch1=4 and ch2=16:  62.16216216216216
Finished Training
Validation accuracy for ch1=4 and ch2=32:  97.29729729729729
Finished Training
Validation accuracy for ch1=4 and ch2=64:  56.75675675675676
Finished Training
Validation accuracy for ch1=8 and ch2=4:  51.351351351351354
Finished Training
Validation accuracy for ch1=8 and ch2=8:  51.351351351351354
Finished Training
Validation accuracy for ch1=8 and ch2=16:  91.89189189189189
Finished Training
Validation accuracy for ch1=8 and ch2=32:  94.5945945945946
Finished Training
Validation accuracy for ch1=8 and ch2=64:  91.89189189189189
Finished Training
Validation accuracy for ch1=16 and ch2=4:  83.78378378378379
Finished Training
Validation accuracy for ch1=16 and ch2=8:  91.89189189189189
Finished Training
Validation accuracy for ch1=16 and ch

In [0]:
# store validation accuracies during training 
accuracies_record = accuracies

# train model with best hyperparameters
net = Net(16,32)
accuracies = []
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
losses = []
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

        inputs, labels = data # data is a list of [inputs, labels]
        optimizer.zero_grad() # zero the parameter gradients

        outputs = net(inputs) # fwd
        l = torch.tensor(labels, dtype=torch.long)
        loss = criterion(outputs, l)
        loss.backward()       # backwd
        optimizer.step()      # optimize

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            losses.append(running_loss / 100)
            running_loss = 0.0
print('Finished Training')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


[1,   100] loss: 7.388
[2,   100] loss: 0.166
[3,   100] loss: 0.140
[4,   100] loss: 0.066
[5,   100] loss: 0.055
Finished Training


In [0]:
with torch.no_grad():
    correct, total = 0, 0
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print("Test accuracy for baseline: ", accuracy)
    accuracies.append(accuracy)

    correct, total = 0, 0
    for data in testloader_african:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print("Test accuracy for African: ", accuracy)
    accuracies.append(accuracy)

    correct, total = 0, 0
    for data in testloader_asian:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print("Test accuracy for Asian: ", accuracy)
    accuracies.append(accuracy)

Test accuracy for baseline:  93.65079365079364
Test accuracy for African:  87.83068783068784
Test accuracy for Asian:  92.95238095238095


## Again, the error rate is lower for the Asian test set than for the African test set, although both are higher than the error rate for the baseline test set.